# Get started with Cymetric to process Cyclus output files

you'll need to install [Cymetric](https://github.com/cyclus/cymetric) and [pandas](https://pandas.pydata.org/) before being able to run this jupyter notebook yourself. Oh, and [Jupyter](https://jupyter.org/), if this is your first experience with it as an interactive computing tool.

First, let's import these packages.

In [1]:
import pandas as pd 
import cymetric as cym

Next, we'll designate one of our minimal output files (which are all named according to their corresponding input file) to process. Let's try the most complicated one, (which is still pretty simple) `source_3_lifetime_sink_1.sqlite` to open as our database

In [2]:
database = cym.dbopen('minimal-input/source_3_lifetime_sink_1.sqlite')

Read the database into an `Evaluator` object to be more computationally efficient. We'll call the evaluator to evaluate any metrics or pull any data from there.

In [3]:
evaler = cym.Evaluator(db=database)

## Pull data into DataFrames

I often use Cymetric to pull the databases I am most interested in so I can do my own work to them as pandas DataFrames. Let's grab the agents (the facilities acting in the simulation) first

In [4]:
agents = evaler.eval('Agents')
agents.head(10)

,SimId,AgentId,Kind,Spec,Prototype,ParentId,Lifetime,EnterTime,ExitTime
0,93a10b68-716b-48cf-b53a-d49bebd5eccc,8,Region,:agents:NullRegion,SingleRegion,-1,-1,0,4.0
1,93a10b68-716b-48cf-b53a-d49bebd5eccc,9,Inst,:agents:NullInst,SingleInstitution,8,-1,0,4.0
2,93a10b68-716b-48cf-b53a-d49bebd5eccc,10,Facility,:cycamore:Source,Source,9,5,0,4.0
3,93a10b68-716b-48cf-b53a-d49bebd5eccc,11,Facility,:cycamore:Source,Source,9,5,0,120.0
4,93a10b68-716b-48cf-b53a-d49bebd5eccc,12,Facility,:cycamore:Source,Source,9,5,0,120.0
5,93a10b68-716b-48cf-b53a-d49bebd5eccc,13,Facility,:cycamore:Sink,Sink,9,-1,0,120.0


Now, we probably care about more than the agents and their lifetimes, so lets pull their transactions. Here instead of just pulling the database transactions (`transactions = evaler.eval('Transactions')`), which doesn't include all the information we may want, we're going to rely on a premade metric that also gives us information on the quantity of material moved in the transaction (information technically housed in the `Resources` database/root metric)

In [5]:
transactions = evaler.eval('TransactionQuantity')
transactions.head(10)

,SimId,TransactionId,ResourceId,ObjId,TimeCreated,SenderId,ReceiverId,Commodity,Units,Quantity
0,93a10b68-716b-48cf-b53a-d49bebd5eccc,0,9,8,0,11,13,commodity,kg,1.0
1,93a10b68-716b-48cf-b53a-d49bebd5eccc,1,11,9,0,10,13,commodity,kg,1.0
2,93a10b68-716b-48cf-b53a-d49bebd5eccc,2,13,10,0,12,13,commodity,kg,1.0
3,93a10b68-716b-48cf-b53a-d49bebd5eccc,3,22,18,1,11,13,commodity,kg,1.0
4,93a10b68-716b-48cf-b53a-d49bebd5eccc,4,24,19,1,10,13,commodity,kg,1.0
5,93a10b68-716b-48cf-b53a-d49bebd5eccc,5,26,20,1,12,13,commodity,kg,1.0
6,93a10b68-716b-48cf-b53a-d49bebd5eccc,6,35,28,2,11,13,commodity,kg,1.0
7,93a10b68-716b-48cf-b53a-d49bebd5eccc,7,37,29,2,10,13,commodity,kg,1.0
8,93a10b68-716b-48cf-b53a-d49bebd5eccc,8,39,30,2,12,13,commodity,kg,1.0
9,93a10b68-716b-48cf-b53a-d49bebd5eccc,9,48,38,3,11,13,commodity,kg,1.0


## Some basic pandas manipulation

Let's say we want to get some information about one of our sources. We know it's from the prototype called source, and for some reason we know it's the last one deployed. So lets pull all the Source prototypes

In [6]:
agents.loc[agents["Prototype"] == "Source"]

,SimId,AgentId,Kind,Spec,Prototype,ParentId,Lifetime,EnterTime,ExitTime
2,93a10b68-716b-48cf-b53a-d49bebd5eccc,10,Facility,:cycamore:Source,Source,9,5,0,4.0
3,93a10b68-716b-48cf-b53a-d49bebd5eccc,11,Facility,:cycamore:Source,Source,9,5,0,120.0
4,93a10b68-716b-48cf-b53a-d49bebd5eccc,12,Facility,:cycamore:Source,Source,9,5,0,120.0


Ok, it's the last one, so its AgentId = 12. The AgentId is important, because there are three agents that have the same exact parameters (they are copies of the prototype Source, but each acts independently). Let's look at the transactions for Source-12.

Transactions are categorized by their sender and their receiver. Source-12 is only a sender, so lets look for transactions where it sent material.

In [7]:
transactions.loc[transactions["SenderId"] == 12]

,SimId,TransactionId,ResourceId,ObjId,TimeCreated,SenderId,ReceiverId,Commodity,Units,Quantity
2,93a10b68-716b-48cf-b53a-d49bebd5eccc,2,13,10,0,12,13,commodity,kg,1.0
5,93a10b68-716b-48cf-b53a-d49bebd5eccc,5,26,20,1,12,13,commodity,kg,1.0
8,93a10b68-716b-48cf-b53a-d49bebd5eccc,8,39,30,2,12,13,commodity,kg,1.0
11,93a10b68-716b-48cf-b53a-d49bebd5eccc,11,52,40,3,12,13,commodity,kg,1.0
14,93a10b68-716b-48cf-b53a-d49bebd5eccc,14,65,50,4,12,13,commodity,kg,1.0
